In [ ]:
!pip install datasets
from datasets import Dataset, DatasetDict, load_dataset, IterableDataset
import re
import torch
import torch.nn as nn
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import RobertaPreTrainedModel, RobertaModel, RobertaTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoConfig
import glob
import os
import pyarrow.parquet as pq
from transformers.modeling_outputs import SequenceClassifierOutput
from google.colab import drive

import torch.nn.functional as F

from transformers.modeling_outputs import ModelOutput
from dataclasses import dataclass


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function preprocessors from CloneDetectionPipeline
class FunctionPreprocessor:
    def get_function(self, code):
        results = []
        fn_list = re.findall(r"\ndef [a-zA-Z0-9_]+\(", code)
        for fn in fn_list:
            results.append(fn[4:-1].strip())
        return results

    def determine_function(self, code, function_name):
        num = len(re.findall(r"[^a-zA-Z]" + function_name + r"[^a-zA-Z]", code))
        return False if num <= 1 else True

    def delete_function(self, code, name):
        start_id, _ = re.search("def " + name, code).span()
        ptr = start_id
        while ptr < len(code) - 1:
            if code[ptr] == "\n" and re.search("[a-zA-Z]", code[ptr + 1]) is not None:
                break
            ptr += 1
        if ptr != len(code) - 1:
            end_id = ptr
            code = code[:start_id] + code[end_id:]
        return code

    def preprocess(self, code):
        code = "\n" + code
        fn_list = self.get_function(code)
        if len(fn_list) == 0:
            return code
        for fn in fn_list:
            flag = self.determine_function(code, fn)
            if not flag:
                code = self.delete_function(code, fn)
        return code

class AnnotationPreprocessor:
    def delete_annotation(self, code):
        sens = code.split("\n")
        sens_processed = [sen.split("#")[0] for sen in sens]  # Remove inline comments
        return "\n".join(sens_processed)

    def delete_import(self, code):
        sens = code.split("\n")
        sens_processed = [sen for sen in sens if "import" not in sen]  # Remove import statements
        return "\n".join(sens_processed)

    def preprocess(self, code):
        code = self.delete_annotation(code)
        code = self.delete_import(code)
        code = re.sub(r"\s+", " ", code).strip()  # Remove excessive whitespace
        return code

In [ ]:
class HuggingFaceCloneDetectionDataset:
    """Custom Iterable Dataset for Hugging Face Trainer"""

    def __init__(self, parquet_loader, function_preprocessor, annotation_preprocessor, tokenizer, max_length=256):
        self.loader = parquet_loader
        self.function_preprocessor = function_preprocessor
        self.annotation_preprocessor = annotation_preprocessor
        self.tokenizer = tokenizer
        self.max_length = max_length

    def preprocess_code(self, code):
        """Apply function & annotation preprocessing"""
        return self.annotation_preprocessor.preprocess(
            self.function_preprocessor.preprocess(code)
        )

    def __iter__(self):
        """Yields dynamically processed & tokenized samples"""
        for batch_df in self.loader:
            for _, row in batch_df.iterrows():
                yield self.process_row(row)

    def process_row(self, row):
        """Preprocesses and tokenizes the two input sequences"""
        code1 = self.preprocess_code(row["code1"])
        code2 = self.preprocess_code(row["code2"])
        label = row["similar"]

        tokens1 = self.tokenizer(code1, max_length=self.max_length, truncation=True, padding="max_length")
        tokens2 = self.tokenizer(code2, max_length=self.max_length, truncation=True, padding="max_length")

        return {
            "input_ids1": tokens1["input_ids"],
            "attention_mask1": tokens1["attention_mask"],
            "input_ids2": tokens2["input_ids"],
            "attention_mask2": tokens2["attention_mask"],
            "labels": label,
        }


class LocalParquetBatchLoader:
    """Memory-efficient batch loader for local Parquet files"""
    def __init__(self, file_paths, batch_size=16):
        self.file_paths = file_paths
        self.batch_size = batch_size
        self.current_file_idx = 0
        self.current_batch_idx = 0
        self.table = None
        self.num_rows = 0

    def load_next_file(self):
        """Loads the next Parquet file into memory in an optimized way"""
        if self.current_file_idx >= len(self.file_paths):
            return None  # No more files

        file_path = self.file_paths[self.current_file_idx]
        print(f"📥 Loading {file_path} from local storage...")

        self.table = pq.read_table(file_path, columns=["code1", "code2", "similar"])
        self.num_rows = self.table.num_rows
        self.current_batch_idx = 0
        self.current_file_idx += 1

    def __iter__(self):
        """Iterates through all files and dynamically loads batches"""
        while self.current_file_idx < len(self.file_paths):
            self.load_next_file()
            if self.table is None:
                break

            while self.current_batch_idx < self.num_rows:
                start = self.current_batch_idx
                end = min(start + self.batch_size, self.num_rows)
                batch = self.table.slice(start, end - start).to_pandas()
                self.current_batch_idx = end
                yield batch




In [ ]:
from itertools import islice

DATASET_PATH = "/content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data"

train_files = sorted(glob.glob(os.path.join(DATASET_PATH, "train-*.parquet")))
val_files = sorted(glob.glob(os.path.join(DATASET_PATH, "val-*.parquet")))

# ✅ Initialize preprocessors and tokenizer
function_preprocessor = FunctionPreprocessor()
annotation_preprocessor = AnnotationPreprocessor()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# ✅ Use LocalParquetBatchLoader instead of GCS loader
train_loader = LocalParquetBatchLoader(train_files, batch_size=128)
val_loader = LocalParquetBatchLoader(val_files, batch_size=128)

train_dataset = IterableDataset.from_generator(lambda: HuggingFaceCloneDetectionDataset(train_loader, function_preprocessor, annotation_preprocessor, tokenizer))
# ✅ Limit val_loader to 5000 samples
LIMIT_EVAL_SAMPLES = 2000

# ✅ Wrap the val_loader to stop after 5000 samples
def limited_val_loader():
    return islice(val_loader, LIMIT_EVAL_SAMPLES)  # Stops after 5000 samples

# ✅ Create a new `val_dataset` with the limited subset
val_dataset = IterableDataset.from_generator(lambda: HuggingFaceCloneDetectionDataset(
    limited_val_loader(), function_preprocessor, annotation_preprocessor, tokenizer
))
# ✅ Check if the dataset loads correctly
for batch in train_dataset:
    print(batch)  # ✅ Ensure it prints one batch
    break



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00000-of-00009.parquet from local storage...
{'input_ids1': [0, 282, 6, 475, 6, 784, 5457, 5456, 1640, 2544, 6, 8135, 49123, 44154, 49338, 10, 5457, 646, 8458, 1640, 32557, 1640, 2544, 6, 8135, 49123, 44154, 43048, 35122, 13, 18134, 11, 1186, 1640, 282, 46077, 741, 5457, 646, 8458, 1640, 32557, 1640, 2544, 6, 8135, 49123, 44154, 43048, 35122, 13, 18134, 11, 1186, 1640, 119, 46077, 740, 5457, 48395, 288, 13, 939, 11, 1186, 1640, 462, 46077, 13, 1236, 11, 1186, 1640, 282, 46077, 13, 939, 11, 1186, 1640, 282, 3256, 13, 1236, 11, 1186, 1640, 462, 3256, 13, 449, 11, 1186, 1640, 119, 3256, 740, 10975, 118, 46386, 267, 742, 49371, 10, 10975, 118, 46386, 330, 742, 1009, 741, 10975, 330, 46386, 267, 742, 13, 939, 11, 1186, 1640, 282, 3256, 5780, 46469, 43809, 26960, 1640, 32557, 1640, 6031, 6, 740, 10975, 118, 742, 47619, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
def siamese_data_collator(features):
    """Custom collator to handle Siamese network inputs"""
    batch = {
        "input_ids1": torch.tensor([f["input_ids1"] for f in features], dtype=torch.long),
        "attention_mask1": torch.tensor([f["attention_mask1"] for f in features], dtype=torch.long),
        "input_ids2": torch.tensor([f["input_ids2"] for f in features], dtype=torch.long),
        "attention_mask2": torch.tensor([f["attention_mask2"] for f in features], dtype=torch.long),
        "labels": torch.tensor([f["labels"] for f in features], dtype=torch.float32),
    }
    return batch

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaPreTrainedModel
from transformers.modeling_outputs import ModelOutput
from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class SiameseOutput(ModelOutput):
    """Custom Hugging Face-compatible output format for similarity models"""
    loss: Optional[torch.FloatTensor] = None
    logits: Optional[torch.FloatTensor] = None
    similarity_score: torch.FloatTensor = None  # ✅ Explicitly named similarity score
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

class SiameseModel(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.training_step = 0  # ✅ Keep track of training steps

        # ✅ Load Pretrained RoBERTa
        self.roberta = RobertaModel(config, add_pooling_layer=False)

        # ✅ Projection Layer to Reduce Embedding Dimensionality
        self.net = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU(),
            nn.Linear(config.hidden_size, config.hidden_size),  # ✅ Project to a smaller feature space
        )

        # # ✅ Loss Functions
        # self.loss_fn_bce = nn.BCEWithLogitsLoss()
        self.loss_fn_cosine = nn.CosineEmbeddingLoss(margin = 0.2)
        self.loss_fn_bce = nn.BCEWithLogitsLoss()

    def last_4_layer_avg(self, model_output):
        layer_indices = [-1, -2, -3, -4]  # Last 4 layers
        stacked_layers = torch.stack([model_output.hidden_states[i] for i in layer_indices])  # (4, batch, seq_len, hidden_dim)
        return stacked_layers.mean(dim=0)[:, 0, :]  # Take mean and use CLS

    def forward(
        self,
        input_ids1=None,
        attention_mask1=None,
        input_ids2=None,
        attention_mask2=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # ✅ Forward pass through RoBERTa
        outputs1 = self.roberta(input_ids1, attention_mask=attention_mask1, return_dict=return_dict)
        outputs2 = self.roberta(input_ids2, attention_mask=attention_mask2, return_dict=return_dict)

        emb1 = self.last_4_layer_avg(outputs1)
        emb2 = self.last_4_layer_avg(outputs2)

        # ✅ Pass through Projection Layer
        emb1 = self.net(emb1)
        emb2 = self.net(emb2)

        # ✅ Normalize embeddings for cosine similarity
        emb1 = F.normalize(emb1, p=2, dim=-1)
        emb2 = F.normalize(emb2, p=2, dim=-1)

        # ✅ Compute Cosine Similarity (Raw Logits)
        cosine_sim = F.cosine_similarity(emb1, emb2, dim=-1).unsqueeze(1)  # Shape: (batch_size, 1)
        logit_scale = 8  # Moderate scale
        margin = 0.2  # Push non-clones further down
        logits = logit_scale * (cosine_sim - margin)
        logits = logits.squeeze(1)


        loss = None
        if labels is not None:
            loss = self.loss_fn_bce(logits, labels.float())  # ✅ Use BCE loss with logits

        return SiameseOutput(
            loss=loss,
            logits=logits,  # ✅ Return logits
            similarity_score=torch.sigmoid(logits),  # ✅ Compute probability from logits
            hidden_states=outputs1.hidden_states if return_dict else None,
            attentions=outputs1.attentions if return_dict else None,
        )





In [ ]:
from transformers import TrainerCallback
import torch

class ClassificationErrorCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        """Computes classification error in real-time on training batches."""
        trainer = kwargs.get("trainer", None)
        if trainer is None or trainer.model is None:
            return

        # Get last training batch
        last_batch = trainer.get_train_dataloader().dataset[-args.per_device_train_batch_size:]
        device = trainer.model.device
        model = trainer.model

        # Move batch to device
        batch = {k: torch.tensor(v).to(device) for k, v in last_batch.items()}

        # Run forward pass
        model.eval()
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits  # ✅ Get logits from SiameseOutput
            labels = batch["labels"]  # ✅ Get ground truth labels

        # Convert logits to probabilities
        probs = torch.sigmoid(logits)

        # Apply threshold (0.5) for classification
        predictions = (probs > 0.5).long()

        # Compute classification error
        errors = (predictions != labels).float().mean().item()

        # Log the classification error
        print(f"🔹 Step {state.global_step}: Classification Error = {errors:.4f}")
        trainer.log({"classification_error": errors})

from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    """Compute classification error instead of loss."""
    logits, labels = eval_pred  # ✅ Unpack logits and labels

    # ✅ Ensure labels are NumPy arrays
    labels = np.array(labels)

    # ✅ Convert logits to probabilities using sigmoid
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # ✅ Convert probabilities to binary predictions (threshold = 0.5)
    preds = (probs > 0.5).astype(int)

    # ✅ Compute classification error (1 - accuracy)
    accuracy = accuracy_score(labels, preds)
    classification_error = 1 - accuracy

    return {"classification_error": classification_error}



In [ ]:
# import json
# from safetensors.torch import load_file
# import torch
# from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
# from transformers import get_linear_schedule_with_warmup


# config = AutoConfig.from_pretrained("roberta-base")
# model = SiameseModel(config)
# model.to("cuda" if torch.cuda.is_available() else "cpu")  # Move to GPU if available

# # ✅ Define warm-up steps (first 1000 batches)
# num_warmup_steps = 600  # Gradually increase LR over 600 batches

# # ✅ 5. Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# # ✅ 6. Set up TrainingArguments
# training_args = TrainingArguments(
#     output_dir="/content/drive/MyDrive/new_siamese_model",
#     logging_dir="/content/drive/MyDrive/new_logs",
#     save_strategy="steps",
#     logging_strategy="steps",
#     logging_steps=200,  # ✅ Logs every 200 steps
#     save_steps=1000,
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=364,
#     num_train_epochs=2,
#     max_steps=320000,
#     load_best_model_at_end=False,
#     report_to="none",
#     logging_first_step=True
# )

# # ✅ 7. Reset optimizer (IGNORE previous optimizer state)
# optimizer = torch.optim.AdamW([
#     {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.01},  # RoBERTa layers
#     {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.01},  # Custom layers
# ], eps=1e-8)
# # ✅ Learning rate scheduler with warm-up

# lr_scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=320000
# )


# # ✅ 9. Initialize Trainer (Manually set optimizer)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=siamese_data_collator,
#     optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
# )


# # ✅ 11. Resume training from the latest model weights, but with a new optimizer
# trainer.train(resume_from_checkpoint=False)  # 🚀 Ignore optimizer mismatch




/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.845300
200,0.707100
400,0.630300
600,0.602400
800,0.603600
1000,0.586200
1200,0.577900
1400,0.579600


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup
# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-1000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ 4. Move model to GPU
model.to(device)
# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 600  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    max_steps=320000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.01},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.01},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=320000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train(resume_from_checkpoint=False)  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.601900
200,0.575200
400,0.572500
600,0.566600
800,0.572900
1000,0.578000
1200,0.577400
1400,0.568000
1600,0.565200
1800,0.562400


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup
# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-2000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ 4. Move model to GPU
model.to(device)
# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 600  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=128,  # ✅ Using larger batch size
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    max_steps=320000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.01},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.01},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=320000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train(resume_from_checkpoint=False)  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.629200
200,0.549400
400,0.542000
600,0.550800
800,0.549000
1000,0.544900
1200,0.545300
1400,0.545300
1600,0.542900
1800,0.544300


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00002-of-00009.parquet from local storage...
📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00003-of-00009.parquet from local storage...


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup
# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-11000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ 4. Move model to GPU
model.to(device)
# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=128,  # ✅ Using larger batch size
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    max_steps=320000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 1e-3, "weight_decay": 0.0001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=30000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train(resume_from_checkpoint=False)  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00003-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.516900
200,0.522600
400,0.527100
600,0.524800
800,0.523700
1000,0.525000
1200,0.519600
1400,0.524400
1600,0.520900
1800,0.519500


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00004-of-00009.parquet from local storage...


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup


config = AutoConfig.from_pretrained("roberta-base")
model = SiameseModel(config)
model.to("cuda" if torch.cuda.is_available() else "cpu")  # Move to GPU if available

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 1000  # Gradually increase LR over 600 batches

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=2000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 1e-3, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=50000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train()  # 🚀 Ignore optimizer mismatch

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,1.827500
200,0.782400
400,0.654200
600,0.607200
800,0.585400
1000,0.538000
1200,0.530000
1400,0.497300


Step,Training Loss
1,1.827500
200,0.782400
400,0.654200
600,0.607200
800,0.585400
1000,0.538000
1200,0.530000
1400,0.497300
1600,0.465800
1800,0.444400


In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup
# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-2000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ 4. Move model to GPU
model.to(device)
# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    max_steps=320000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train(resume_from_checkpoint=False)  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.764500
200,0.527700
400,0.485800
600,0.444100
800,0.429200
1000,0.436200
1200,0.444100
1400,0.427100
1600,0.413100
1800,0.430600


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-7000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

config = AutoConfig.from_pretrained("roberta-base", output_hidden_states=True)

model = SiameseModel(config)
model.to("cuda" if torch.cuda.is_available() else "cpu")  # Move to GPU if available

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    max_steps=320000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train()  # 🚀 Ignore optimizer mismatch

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,1.839000
200,0.733800
400,0.581200
600,0.504300
800,0.453000
1000,0.439400
1200,0.424100
1400,0.411700
1600,0.396200
1800,0.377500


In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-3000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler)  # 🚀 Reset optimizer state, Ignore previous optimizer
)


# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train()  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.412300
200,0.270500
400,0.282100
600,0.273500
800,0.273600
1000,0.281300
1200,0.280600
1400,0.276600
1600,0.271600
1800,0.281300


KeyboardInterrupt: 

In [ ]:
import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-3000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,
    save_strategy="steps",
    logging_strategy="steps",
    evaluation_strategy="steps",  # ✅ Evaluate every few steps
    eval_steps=1000,  # ✅ Log classification error every 1000 steps
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)



# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ Now only evaluates on 5000 samples!
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,  # ✅ Use classification error instead of loss!
)



# ✅ 11. Resume training from the latest model weights, but with a new optimizer
trainer.train()  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00002-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.372900
200,0.226600
400,0.238700
600,0.226000
800,0.225900
1000,0.215400
1200,0.218900
1400,0.213200
1600,0.209700
1800,0.205200


KeyboardInterrupt: 

In [ ]:

import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-3000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 2e-4, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ Now only evaluates on 1000 samples!
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler),
)


trainer.train()  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.322600
200,0.190000
400,0.196500
600,0.192600
800,0.186500
1000,0.193900
1200,0.194000
1400,0.189000
1600,0.184600
1800,0.195700


KeyboardInterrupt: 

In [ ]:

import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-3000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=64,  # ✅ Using larger batch size
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 5e-5, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ Now only evaluates on 1000 samples!
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler),
)


trainer.train()  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00002-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.235500
200,0.180300
400,0.187200
600,0.175100
800,0.171500
1000,0.171000
1200,0.174600
1400,0.169500
1600,0.168600
1800,0.163800


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00003-of-00009.parquet from local storage...
📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00004-of-00009.parquet from local storage...


KeyboardInterrupt: 

In [ ]:

import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-16000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=128,  # ✅ Using larger batch size
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 5e-5, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ Now only evaluates on 1000 samples!
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler),
)


trainer.train()  # 🚀 Ignore optimizer mismatch

🔹 Missing keys: []
🔹 Unexpected keys: []


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00001-of-00009.parquet from local storage...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.145600
200,0.110600
400,0.110600
600,0.113600
800,0.110600
1000,0.116400
1200,0.109900
1400,0.111100
1600,0.108600
1800,0.112200


📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00002-of-00009.parquet from local storage...
📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00003-of-00009.parquet from local storage...
📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00004-of-00009.parquet from local storage...
📥 Loading /content/drive/My Drive/4YPdataset/1-fold-clone-detection-600k-5fold/data/train-00005-of-00009.parquet from local storage...


KeyboardInterrupt: 

In [ ]:

import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup


import torch
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
from transformers import AutoTokenizer

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-20000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded
model.to(device)
model.eval()  # Put model in evaluation mode

# ✅ Set up dataset loader
file_paths = ["//content/drive/MyDrive/4YPdataset/1-fold-clone-detection-600k-5fold/data/val-00000-of-00003.parquet"]  # Replace with actual Parquet file paths
batch_loader = LocalParquetBatchLoader(file_paths, batch_size=64)
# ✅ Initialize preprocessors and tokenizer
function_preprocessor = FunctionPreprocessor()
annotation_preprocessor = AnnotationPreprocessor()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# ✅ Track performance metrics
total_samples = 14400
processed_samples = 0
total_correct = 0
total_batches = 0

# ✅ Process dataset in batches
with torch.no_grad():  # Disable gradient calculation for faster inference
    for batch in tqdm(batch_loader, desc="Evaluating model"):
        if processed_samples >= total_samples:
            break  # Stop after 3200 samples

        # ✅ Convert DataFrame to input dictionary
        input_data = HuggingFaceCloneDetectionDataset(None, function_preprocessor, annotation_preprocessor, tokenizer)
        batch_dicts = [input_data.process_row(row) for _, row in batch.iterrows()]

        # ✅ Convert to PyTorch tensors & move everything to `device`
        batch_inputs = {
            "input_ids1": torch.tensor([d["input_ids1"] for d in batch_dicts], dtype=torch.long).to(device),
            "attention_mask1": torch.tensor([d["attention_mask1"] for d in batch_dicts], dtype=torch.long).to(device),
            "input_ids2": torch.tensor([d["input_ids2"] for d in batch_dicts], dtype=torch.long).to(device),
            "attention_mask2": torch.tensor([d["attention_mask2"] for d in batch_dicts], dtype=torch.long).to(device),
        }
        labels = torch.tensor([d["labels"] for d in batch_dicts], dtype=torch.float).to(device)  # ✅ Move to `device`

        # ✅ Run model inference
        outputs = model(**batch_inputs)
        similarity_scores = outputs.similarity_score.cpu().numpy()  # Sigmoid output
        logits = outputs.logits.cpu().numpy()  # Raw logit values
        preds = (similarity_scores >= 0.5).astype(int)  # Convert to binary labels

        # ✅ Compute batch classification error
        batch_correct = (preds == labels.cpu().numpy()).sum()
        batch_total = labels.shape[0]
        batch_error = 1 - (batch_correct / batch_total)

        # ✅ Update overall metrics
        total_correct += batch_correct
        processed_samples += batch_total
        total_batches += 1

        # ✅ Print batch results
        print(f"\n🔹 **Batch {total_batches}:**")
        print(f"   ✅ Avg Similarity Score: {similarity_scores.mean():.4f}")
        print(f"   ✅ Max Logits: {logits.max():.4f}, Min Logits: {logits.min():.4f}")
        print(f"   ❌ Batch Classification Error: {batch_error:.4%}")

# ✅ Final Evaluation Summary
overall_error = 1 - (total_correct / processed_samples)
print("\n🔹 **Final Model Evaluation:**")
print(f"   🔹 Total Samples Processed: {processed_samples}")
print(f"   🔹 Total Batches: {total_batches}")
print(f"   ✅ Overall Classification Error: {overall_error:.4%}")


🔹 Missing keys: []
🔹 Unexpected keys: []


Evaluating model: 0it [00:00, ?it/s]

📥 Loading //content/drive/MyDrive/4YPdataset/1-fold-clone-detection-600k-5fold/data/val-00000-of-00003.parquet from local storage...


Evaluating model: 1it [00:00,  1.35it/s]


🔹 **Batch 1:**
   ✅ Avg Similarity Score: 0.4616
   ✅ Max Logits: 6.4000, Min Logits: -2.0648
   ❌ Batch Classification Error: 20.3125%


...


🔹 **Batch 223:**
   ✅ Avg Similarity Score: 0.3960
   ✅ Max Logits: 6.2612, Min Logits: -2.2133
   ❌ Batch Classification Error: 12.5000%


Evaluating model: 224it [01:46,  2.13it/s]


🔹 **Batch 224:**
   ✅ Avg Similarity Score: 0.5454
   ✅ Max Logits: 6.3799, Min Logits: -2.6398
   ❌ Batch Classification Error: 14.0625%


Evaluating model: 225it [01:46,  2.11it/s]


🔹 **Batch 225:**
   ✅ Avg Similarity Score: 0.3744
   ✅ Max Logits: 6.3509, Min Logits: -2.2505
   ❌ Batch Classification Error: 20.3125%

🔹 **Final Model Evaluation:**
   🔹 Total Samples Processed: 14400
   🔹 Total Batches: 225
   ✅ Overall Classification Error: 19.1875%


In [ ]:

import json
from safetensors.torch import load_file
import torch
from transformers import AutoConfig, AutoTokenizer, Trainer, TrainingArguments, AdamW, TrainerState
from transformers import get_linear_schedule_with_warmup

# ✅ 1. Load model config
model_checkpoint = "/content/drive/MyDrive/new_siamese_model/checkpoint-20000"
config = AutoConfig.from_pretrained(model_checkpoint)
model = SiameseModel(config)

# ✅ 2. Load model weights from `safetensors`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights = load_file(f"{model_checkpoint}/model.safetensors", device="cpu")

# ✅ 3. Apply weights correctly
missing_keys, unexpected_keys = model.load_state_dict(model_weights, strict=False)
print(f"🔹 Missing keys: {missing_keys}")  # Should be empty if properly loaded
print(f"🔹 Unexpected keys: {unexpected_keys}")  # Should be empty if properly loaded

# ✅ Define warm-up steps (first 1000 batches)
num_warmup_steps = 200  # Gradually increase LR over 600 batches

# ✅ 5. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/new_siamese_model",
    per_device_train_batch_size=128,  # ✅ Using larger batch size
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    max_steps=50000,
    fp16=True,  # ✅ Enables mixed precision training
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=200,
    save_steps=1000,
    load_best_model_at_end=False,
    report_to="none",
    logging_first_step=True
)


# ✅ 7. Reset optimizer (IGNORE previous optimizer state)
optimizer = AdamW([
    {"params": model.roberta.parameters(), "lr": 2e-5, "weight_decay": 0.0001},  # RoBERTa layers
    {"params": model.net.parameters(), "lr": 5e-5, "weight_decay": 0.001},  # Custom layers
], eps=1e-8)
# ✅ Learning rate scheduler with warm-up

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=40000
)

# ✅ 9. Initialize Trainer (Manually set optimizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ Now only evaluates on 1000 samples!
    data_collator=siamese_data_collator,
    optimizers=(optimizer, lr_scheduler),
)


trainer.train()  # 🚀 Ignore optimizer mismatch